In [1]:
import numpy 
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge, Rectangle
from matplotlib.path import Path

In [2]:
import tensorflow as tf

/Users/samanthausman/anaconda2/envs/ML-practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/samanthausman/anaconda2/envs/ML-practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/samanthausman/anaconda2/envs/ML-practice/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/

ImportError: cannot import name 'export_saved_model'

### Creating Shapes

In [ ]:
def _patch2im(thepatch, imshape):
    # Create meshgrid of coordinates
    y = numpy.arange(1, imshape[0])
    x = numpy.arange(1, imshape[1])
    g = numpy.meshgrid(x, y)

    coords = numpy.array(list(zip(*(c.flat for c in g))))

    # Create path
    elpath = Path(thepatch.get_verts())

    # Determine all points in the grid ithat are in the path.
    validcoords = elpath.contains_points(coords)
    ellipsepoints = coords[validcoords]

    outim = numpy.zeros(imshape)
    outim[ellipsepoints[:, 0], ellipsepoints[:, 1]] = 1

    return outim


def create_polygon(imshape, xy, width, height, angle=0.0, fill=True):

    # Create the arc patch we want to add to the image
    rect = Rectangle(xy, width, height, angle=0.0, fill=fill)

    return _patch2im(rect, imshape)


def create_circle(imshape, center, radius, width):
    """
    Create a cirlce.
    """

    # Create the arc patch we want to add to the image
    return create_arc(imshape, center, radius, width=width, theta1=0, theta2=360)
  
def create_line(imshape, center, radius, width):
    """
    Create a cirlce.
    """

    # Create the arc patch we want to add to the image
    # return create_line(imshape, center, radius, width=width, theta1=0, theta2=359)


def create_arc(imshape, center, radius, width, theta1, theta2):
    """
    Create an arc (outline of a circle) with a specified
    thickness (width).
    """

    # Create the arc patch we want to add to the image
    arc = Wedge(center, radius, theta1, theta2, width)

    return _patch2im(arc, imshape)

In [ ]:
imblah = numpy.zeros((192, 192))

# Example arc
arc = create_arc(imblah.shape, center=(150, 128), radius=56, width=6, theta1=27, theta2=156)
imblah += arc * 35

# Example filled circle
circle = create_circle(imblah.shape, center=(50, 28), radius=56, width=56)
imblah += circle * 21

# Example unfilled circle
circle = create_circle(imblah.shape, center=(28, 200), radius=50, width=8)
imblah += circle * 61

# Example polygon
rect = create_polygon(imblah.shape, (150, 28), width=27, height=44)
imblah += rect * 13

In [ ]:
plt.imshow(imblah)

### Loop to create set of circle images

In [ ]:
num_circ = 10000
radii = numpy.random.randint(10, 50, num_circ)
x_coord_circ = numpy.random.randint(50, 142, num_circ)
y_coord_circ = numpy.random.randint(50, 142, num_circ)
amplitude_circ = numpy.ones((num_circ))# numpy.random.randint(0, 100, num_circ)

In [ ]:
circles = numpy.zeros((num_circ, 192, 192))

for i in range(0, num_circ): 
    circle = create_circle(circles[i].shape, center=(x_coord_circ[i], y_coord_circ[i]), radius=radii[i], width=radii[i])
    circles[i] += circle * amplitude_circ[i]

In [ ]:
plt.imshow(circles[55])
plt.colorbar()

### Loop to create set of rectangle images

In [ ]:
num_rect = 10000
width = numpy.random.randint(20, 50, num_rect)
height = numpy.random.randint(20, 50, num_rect)
amplitude_rect = numpy.ones((num_rect)) # numpy.random.randint(0, 100, num_rect)
x_coord_rect = numpy.random.randint(50, 142, num_rect)
y_coord_rect = numpy.random.randint(50, 142, num_rect)
#Make squares
#Don't let rectangles clip edge
#Same amplitude

In [ ]:
rectangles = numpy.zeros((num_rect, 192, 192))

for j in range(0, num_rect):
    rect = create_polygon(rectangles[j].shape, (x_coord_rect[j], y_coord_rect[j]), width=width[j], height=height[j])
    rectangles[j] += rect * amplitude_rect[j]

In [ ]:
plt.imshow(rectangles[99])
plt.colorbar()

In [ ]:
#Setting percentage of images set aside for training
train_percent = 0.8
index_cutoff = int(0.8*num_circ)

In [ ]:
train_circles = circles[:index_cutoff]
validation_circles = circles[index_cutoff:]

In [ ]:
train_rectangles = rectangles[:index_cutoff]
validation_rectangles = rectangles[index_cutoff:]

In [ ]:
#add these together using np.vstack or hstack or whatever
train = numpy.vstack((train_circles, train_rectangles))
validation = numpy.vstack((validation_circles, validation_rectangles))

labels_circ_train = numpy.zeros((numpy.shape(train_circles))[0])
labels_rect_train = numpy.ones((numpy.shape(train_rectangles))[0])
labels_circ_valid = numpy.zeros((numpy.shape(validation_circles))[0])
labels_rect_valid = numpy.ones((numpy.shape(validation_rectangles))[0])

In [ ]:
train_labels = numpy.hstack((labels_circ_train, labels_rect_train))
validation_labels = numpy.hstack((labels_circ_valid, labels_rect_valid))

#Make label array (0 for circle, 1 for rectangle)
#Make corresponding dictionary for each label
#Refer to https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
#Also good: https://visualstudiomagazine.com/articles/2020/09/10/pytorch-dataloader.aspx

## Loading Data into TensorFlow

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((validation, validation_labels))

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

## Build, Compile and Train Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(192, 192)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(train_dataset, epochs=10)

## Running on Validation Set

In [ ]:
results = model.evaluate(test_dataset)

## Old Code Attempts with PyTorch